In [1]:
from obj import *
import math
import pandas as pd
# from EnergyPlusKernel import *


# Mapeamento

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:


# --- CARREGANDO OS DADOS DAS SUAS PLANILHAS ---

try:

 

    df_paredes = pd.read_excel("dados.xlsx", sheet_name="comb_paredes")
    
    df_radier = pd.read_excel("dados.xlsx", sheet_name="comb_radier")
    


    # Limpeza e preparação dos dados de custo
    # Converte a coluna de custo para numérico, substituindo vírgula por ponto
    # df_paredes['custo_final'] = df_paredes['custo(m2)'].str.replace(',', '.', regex=False).astype(float)
    # df_radier['custo_final'] = df_radier['custo(m2)'].str.replace(',', '.', regex=False).astype(float)
    
    # Para cobertura e vidros, vamos assumir um custo por enquanto
    # Você precisará adicionar a coluna de custo real na sua planilha 'comb_cobertura.csv'
    # if 'custo' not in df_cobertura.columns:
    #     df_cobertura['custo_final'] = 100.0 # Custo placeholder
   


    # --- CRIANDO O MAPEAMENTO DINAMICAMENTE ---

    MAPEAMENTO_E_CUSTOS = {
        'paredes': [{'nome': f"Parede_Opcao_{i}", 'custo_m2': row.custo_final, 'layers': row.to_dict()} for i, row in df_paredes.iterrows()],
        'radier': [{'nome': f"Radier_Opcao_{i}", 'custo_m2': row.custo_final, 'layers': row.to_dict()} for i, row in df_radier.iterrows()],
        
    }

    # Você também precisará das áreas para o cálculo de custo
    AREAS = {
        'paredes': 300,   # m² (exemplo)
        'radier': 150,    # m² (exemplo)
        
    }

    print("Mapeamento de materiais criado com sucesso!")
    print(f"Número de opções de paredes: {len(MAPEAMENTO_E_CUSTOS['paredes'])}")
    
    print(f"Número de opções de radier: {len(MAPEAMENTO_E_CUSTOS['radier'])}")
    


except FileNotFoundError as e:
    print(f"Erro: Arquivo não encontrado. Verifique se o nome e o caminho estão corretos.")
    print(e)

Mapeamento de materiais criado com sucesso!
Número de opções de paredes: 98
Número de opções de radier: 5


In [4]:

# from EnergyPlusKernel import *

# Definindo problema

In [4]:
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.operators.crossover.pntx import TwoPointCrossover
from pymoo.operators.mutation.pm import PolynomialMutation
from pymoo.optimize import minimize
from energyplus2 import * # Importa suas funções
import time

class EdificacaoProblem(Problem):

    def __init__(self, mapeamento, areas):
        self.mapeamento = mapeamento
        self.areas = areas

        # Definindo o número de variáveis e seus limites a partir dos dados carregados
        self.componentes = list(mapeamento.keys())
        n_var = len(self.componentes)
        xu = [len(mapeamento[comp]) - 1 for comp in self.componentes]

        super().__init__(n_var=n_var,
                         n_obj=2,
                         n_constr=0,
                         xl=np.zeros(n_var),
                         xu=np.array(xu),
                         vtype=int)

    def _evaluate(self, X, out, *args, **kwargs):
        all_custos = []
        all_confortos_neg = []

        # X é um array de soluções. Ex: [[parede_idx1, cob_idx1, rad_idx1, vidro_idx1], [parede_idx2, ...]]
        for x_solucao in X:
            
            # --- 1. Decodificar e Calcular Custo ---
            custo_total = 0
            materiais_escolhidos = {}
            
            for i, comp_nome in enumerate(self.componentes):
                idx_escolhido = int(round(x_solucao[i]))
                escolha = self.mapeamento[comp_nome][idx_escolhido]
                
                materiais_escolhidos[comp_nome] = escolha['nome']
                custo_total += escolha['custo_m2'] * self.areas.get(comp_nome, 0)
            
            # --- 2. Preparar e Rodar Simulação ---
            df_conbinacoes = (df_paredes, df_radier)
            indices_escolhidos = tuple(int(round(x)) for x in x_solucao)
            caminho_idf_modificado, flag = configurar_construction("ultimo_idf_modificado_modificado_material.idf", df_conbinacoes, indices_escolhidos)
            # caminho_idf_modificado = "ultimo_idf_modificado_modificado_material_modificado_construction.idf"


            # with open(caminho_idf_modificado, 'r', encoding='utf-8') as f:
            #     conteudo_idf = f.read()
            #     with open("ultimo_idf_modificado.txt", 'w', encoding='utf-8') as f2:
            #         f2.write(conteudo_idf)


            # (Esta parte ainda é um placeholder e precisa da sua implementação final)
            print(f"Avaliando solução: {materiais_escolhidos} com Custo: R${custo_total:.2f}")

            
            if not flag:
                print(f"!!! ERRO na configuração do IDF para a solução {x_solucao}. Penalizando.")
                custo_total = 999999999
                conforto_horas = 0

                all_custos.append(custo_total)
                all_confortos_neg.append(-conforto_horas) # Maximizar conforto => Minimizar (-conforto)
                continue

            processar_idf("simulacao","Zona 6", caminho_ep="C:/EnergyPlusV24-1-0/energyplus.exe",caminho_epw="BRA_GO_Catalao.867770_INMET.epw", caminho_idf=caminho_idf_modificado)
                
            
            

            all_custos.append(custo_total)
            all_confortos_neg.append(-conforto_horas) # Maximizar conforto => Minimizar (-conforto)

        out["F"] = np.column_stack([all_custos, all_confortos_neg])

# config idf

In [6]:
# caminho_energyplus_sua_maquina = "C:/EnergyPlusV24-1-0/energyplus.exe"
# entrada_idf = "versaoTESTE23base copy.idf"
# epw_file = "BRA_GO_Catalao.867770_INMET.epw"
# material = "mats.xlsx"
# vidros = "glass.xlsx"


# idf_v = alterar_versao_energyplus_idf(entrada_idf)
# idf_vs = configurar_simulation_control(idf_v)
# clean_idf(idf_v)
# idf_vsb = configurar_building(idf_vs)
# clean_idf(idf_vs)
# idf_vsbt = configurar_timestep(idf_vsb)
# clean_idf(idf_vsb)
# idf_vsbtr = configurar_run_period(idf_vsbt)
# clean_idf(idf_vsbt)
# idf_vsbtrd = configurar_dias_especiais(idf_vsbtr)
# clean_idf(idf_vsbtr)
# idf_vsbtrdm = adicionar_materiais_do_excel(idf_vsbtrd, material)
# clean_idf(idf_vsbtrd)
# idf_vsbtrdmv = adicionar_vidros_do_excel(idf_vsbtrdm, vidros)
# clean_idf(idf_vsbtrdm)
# idf_final = configurar_localizacao_do_epw(idf_vsbtrdmv, epw_file)
# clean_idf(idf_vsbtrdmv)
# # idf = salvar_idf_final(entrada_idf, idf_final)
# # clean_idf(idf_final)


# Config otmizador

In [7]:
# --- CONFIGURANDO E RODANDO O OTIMIZADOR ---

# Verifica se o mapeamento foi carregado antes de continuar
if 'MAPEAMENTO_E_CUSTOS' in locals():
    problem = EdificacaoProblem(MAPEAMENTO_E_CUSTOS, AREAS)

    algorithm = NSGA2(
        pop_size=10,  # Mantenha baixo para testes iniciais
        sampling=IntegerRandomSampling(),
        crossover=TwoPointCrossover(),
        mutation=PolynomialMutation(),
        eliminate_duplicates=True
    )

    res = minimize(problem,
                   algorithm,
                   ('n_gen', 5), # Mantenha baixo para testes iniciais
                   seed=1,
                   verbose=True)

    # --- ANALISANDO OS RESULTADOS ---
    print("\n" + "="*50)
    print("OTIMIZAÇÃO CONCLUÍDA")
    print("="*50)

    # Inverta o sinal do conforto de volta para positivo
    res.F[:, 1] *= -1

    print("\nMelhores combinações encontradas (Fronteira de Pareto):")
    for i in range(len(res.X)):
        solucao_indices = res.X[i]
        solucao_custo = res.F[i, 0]
        solucao_conforto = res.F[i, 1]
        
        print(f"\n--- Solução {i+1} ---")
        print(f"  > Custo: R$ {solucao_custo:,.2f} | Conforto: {solucao_conforto:,.2f} horas")
        print(f"  > Combinação de Índices: {solucao_indices}")
        # Decodificando os índices para nomes para fácil leitura
        for j, comp_nome in enumerate(problem.componentes):
            nome_mat = MAPEAMENTO_E_CUSTOS[comp_nome][round(solucao_indices[j])]['nome']
            print(f"    - {comp_nome.title()}: {nome_mat} (Índice {solucao_indices[j]})")

ValueError: too many values to unpack (expected 2)

In [8]:
from energyplus2 import *
import os

simulacao = "simulacao"
if not os.path.exists(simulacao):
    os.makedirs(simulacao)

epw = "BRA_GO_Catalao.867770_INMET.epw"
idf = "ultimo_idf_modificado_modificado_material_modificado_construction.idf"
caminho_ep = "C:/EnergyPlusV24-1-0/energyplus.exe"

# dict = processar_idf(simulacao, "Zona 6", caminho_ep, epw, idf)

executar_energyplus(caminho_ep,simulacao,epw,idf)

Rodando comando: C:/EnergyPlusV24-1-0/energyplus.exe -r -d simulacao -w BRA_GO_Catalao.867770_INMET.epw --expandobjects --readvars ultimo_idf_modificado_modificado_material_modificado_construction.idf


'simulacao\\ultimo_idf_modificado_modificado_material_modificado_construction.csv'

In [3]:
import pandas as pd
from energyplus2 import *

idf = "ultimo_idf_modificado.idf"
material = pd.read_excel("dados.xlsx", sheet_name="props_mats")

# material['Espessura (m)'] = material['Espessura (m)'].astype(str).str.replace(',','.').astype(float)
# material[['Absortância Térmica','Absortância Solar', 'Absortância Visivel']] = material[['Absortância Térmica','Absortância Solar', 'Absortância Visivel']].astype(str).replace('-1989','0', regex=True).astype(float)

# display(material)
# material.to_excel("materiais.xlsx", index=False)

adicionar_materiais_do_excel(idf, "materiais.xlsx")



'ultimo_idf_modificado_modificado_material.idf'

In [ ]:
print(dict)

{'QUARTO': 7488.0, 'TELHADO': 5077.0}


In [ ]:
os.path.isfile("versaoTESTE23base copy_modificado_versao_modificado_simulation_control_modificado_building_modificado_timestep_modificado_run_period_modificado_feriados_modificado_material_modificado_glass_modificado_localizacao_epw_modificado_construction.idf")

True